## Data Cleaning - Assessing Text Language

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import email
import warnings 
warnings.filterwarnings('ignore')
import parse
import re
import math

In [2]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
df=pd.read_csv('/Users/kshitijmittal/Documents/UChicago Acad/Hedwig_AI/Hedwig/00_Data/cleaned_emails_dataset.csv')
df.head()

,file,message,Date,Subject,X-From,X-To,X-Folder,body,Employee
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,14-05-2001 16:39:00,NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,'sent mail,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,04-05-2001 13:51:00,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,'sent mail,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,18-10-2000 03:00:00,Re: test,Phillip K Allen,Leah Van Arsdall,'sent mail,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,23-10-2000 06:13:00,NaN,Phillip K Allen,Randall L Gay,'sent mail,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,31-08-2000 05:07:00,Re: Hello,Phillip K Allen,Greg Piper,'sent mail,Let's shoot for Tuesday at 11:45.,allen-p


In [6]:
df.head(10)

,Subject,X-Folder,body
0,NaN,'sent mail,Here is our forecast\n\n
1,Re:,'sent mail,Traveling to have a business meeting takes the...
2,Re: test,'sent mail,test successful. way to go!!!
3,NaN,'sent mail,"Randy,\n\n Can you send me a schedule of the s..."
4,Re: Hello,'sent mail,Let's shoot for Tuesday at 11:45.
5,Re: Hello,'sent mail,"Greg,\n\n How about either next Tuesday or Thu..."
6,NaN,'sent mail,Please cc the following distribution list with...
7,Re: PRC review - phone calls,'sent mail,any morning between 10 and 11:30
8,Re: High Speed Internet Access,'sent mail,1. login: pallen pw: ke9davis\n\n I don't thi...
9,FW: fixed forward or other Collar floor gas pr...,'sent mail,---------------------- Forwarded by Phillip K ...


In [4]:
df[['body']]

,body
0,Here is our forecast\n\n
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n\n Can you send me a schedule of the s..."
4,Let's shoot for Tuesday at 11:45.
...,...
517396,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,Some of my position is with the Alberta Term b...
517398,2\n\n -----Original Message-----\nFrom: \tDouc...
517399,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...


In [5]:
df[['Subject']]

,Subject
0,NaN
1,Re:
2,Re: test
3,NaN
4,Re: Hello
...,...
517396,Trade with John Lavorato
517397,Gas Hedges
517398,RE: CONFIDENTIAL
517399,Calgary Analyst/Associate


----
## Checking the type of emails in our dataset

In [12]:
df['Subject'].value_counts()

RE:                                                      6477
Re:                                                      6306
Demand Ken Lay Donate Proceeds from Enron Stock Sales    1124
FW:                                                       938
Schedule Crawler: HourAhead Failure                       900
                                                         ... 
RE: Citigroup                                               1
FW: Proposed Retention Bonuses for Netco                    1
Master Plan                                                 1
RE: Bonus and Options                                       1
Calgary Analyst/Associate                                   1
Name: Subject, Length: 159289, dtype: int64

In [17]:
pattern=r'\b(RE:|re:|Re:)'
text="RE: Bonus and Options 1"
re.match(pattern,text)

<re.Match object; span=(0, 3), match='RE:'>

In [38]:
def email_type(x):
    pattern_re=r'\b(RE:|re:|Re:)'
    pattern_fw=r'\b(FW:|fw:|Fw:)'
    if pd.isna(x):
        return "no_subject"
    if re.match(pattern_re, str(x)):
        return "reply"
    elif re.match(pattern_fw, str(x)):
        return "forward"
    else:
        return "new_email"

In [39]:
df['email_type']=df['Subject'].apply(lambda x: email_type(x))

In [40]:
df[['Subject','email_type']]

,Subject,email_type
0,NaN,no_subject
1,Re:,reply
2,Re: test,reply
3,NaN,no_subject
4,Re: Hello,reply
...,...,...
517396,Trade with John Lavorato,new_email
517397,Gas Hedges,new_email
517398,RE: CONFIDENTIAL,reply
517399,Calgary Analyst/Associate,new_email


In [41]:
df['email_type'].value_counts()

new_email     312995
reply         153217
forward        32002
no_subject     19187
Name: email_type, dtype: int64

----
## Cleaning Text Body